In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get started with AutoML Training

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/get_started_automl_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
        <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/get_started_automl_training.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
        </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/get_started_automl_training.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use AutoML in production. This tutorial covers get started with AutoML training.

Learn more about [AutoML training](https://cloud.google.com/vertex-ai/docs/training-overview).

### Objective

In this tutorial, you learn how to use `AutoML` for training with `Vertex AI`.

This tutorial uses the following Google Cloud ML services:

- `AutoML Training`
- `Vertex AI Datasets`

The steps performed include:

- Train an image model
- Export the image model as an edge model
- Train a tabular model
- Export the tabular model as a cloud model
- Train a text model
- Train a video model

### Recommendations

When doing E2E MLOps on Google Cloud, the following are best practices for when to use AutoML:

* **You have a limited amount of training data**

* **You want to establish a baseline metric before experimenting with a custom model**

### Datasets

#### Image

The image dataset used for this tutorial is the [Flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) from [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview). The version of the dataset in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the type of flower in a given image from a class of five flowers: daisy, dandelion, rose, sunflower, or tulip.

#### Tabular

The tabular dataset used for this tutorial is the GSOD dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). The version of the dataset you use only the fields year, month and day to predict the value of mean daily temperature (mean_temp).

#### Text

The text dataset used for this tutorial is the [Happy Moments dataset](https://www.kaggle.com/ritresearch/happydb) from [Kaggle Datasets](https://www.kaggle.com/ritresearch/happydb). The version of the dataset you use in this tutorial is stored in a public Cloud Storage bucket.

#### Video

The video dataset used for this tutorial is the golf swing recognition portion of the [Human Motion dataset](https://todo) from [MIT](http://cbcl.mit.edu/publications/ps/Kuehne_etal_iccv11.pdf). The version of the dataset you use in this tutorial is stored in a public Cloud Storage bucket. The trained model predicts the start frame where a golf swing begins.

### Costs
This tutorial uses billable components of Google Cloud:

- Vertex AI
- Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage pricing](https://cloud.google.com/storage/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Installations

Install the packages required for executing this notebook.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

# Install the packages

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG -q
! pip3 install --upgrade google-cloud-storage $USER_FLAG -q

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI, Compute Engine and Cloud Storage APIs](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage_component).

1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using Vertex AI Workbench Notebooks**, your environment is already authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

1. **Click Create service account**.

2. In the **Service account name** field, enter a name, and click **Create**.

3. In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex AI" into the filter box, and select **Vertex AI Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

4. Click Create. A JSON file that contains your key downloads to your local environment.

5. Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = False
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        IS_COLAB = True
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_URI == "" or BUCKET_URI is None or BUCKET_URI == "gs://[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP
    BUCKET_URI = "gs://" + BUCKET_NAME

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [1]:
import google.cloud.aiplatform as aiplatform

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## AutoML training job

AutoML can be used to automatically train a wide variety of image model types. AutoML automates the following:

- Dataset preprocessing
- Feature Engineering
- Data feeding
- Model Architecture selection
- Hyperparameter tuning
- Training the model

Learn more about [Vertex AI for AutoML users](https://cloud.google.com/vertex-ai/docs/start/automl-users)

## AutoML image models

AutoML can train the following types of image models:

- classification
- objection detection
- segmentation

A model can be trained for either deployment to the cloud or exported to the edge.

Learn more about [AutoML Model Types](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)

### Data preparation

The Vertex `Dataset` resource for images has some requirements for your data:

- Images must be stored in a Cloud Storage bucket.
- Each image file must be in an image format (PNG, JPEG, BMP, ...).
- There must be an index file stored in your Cloud Storage bucket that contains the path and label for each image.
- The index file must be either CSV or JSONL.

Learn more about [Preparing image data](https://cloud.google.com/vertex-ai/docs/datasets/prepare-image)

#### CSV

For image classification, the CSV index file has the requirements:

- No heading.
- First column is the Cloud Storage path to the image.
- Second column is the label.
- Any remaining columns are additional labels for multi-label image classification.

For image object detection, the CSV index file has the requirements:

- No heading.
- First column is the Cloud Storage path to the image.
- Second column is the label.
- Third/Fourth columns are the upper left corner of bounding box. Coordinates are normalized, between 0 and 1.
- Fifth/Sixth/Seventh columns are not used and should be 0.
- Eighth/Ninth columns are the lower right corner of the bounding box.

##### ML_USE

Each row may additionally specify which split to assign the data item to when the dataset is split for training; otherwise, the dataset will be randomly split: 80/10/10.

The `ml_use` assignment is specified by prepending a column for specifying the assignment -- as the first column. The value may be one of: training, test, or validation.

#### JSONL

For image classification, the JSONL index file has the requirements:

- Each data item is a separate JSON object, on a separate line.
- The key/value pair `image_gcs_uri` is the Cloud Storage path to the image.
- The key/value pair `display_name` is the label for the image.

    { 'image_gcs_uri': image, 
      'classification_annotations': 
          { 'display_name': label
          }
    }
    
For multi-label, the labels are specified as a list of `display_name` key/value pairs:

    { 'image_gcs_uri': image, 
      'classification_annotations': [
          { 'display_name': label1
          },
          { 'display_name': labelN
          },
       ]
    }
    
For object detection, the JSONL index file has the requirements:

- Each data item is a separate JSON object, on a separate line.
- The key/value pair `image_gcs_uri` is the Cloud Storage path to the image.
- The key/value pair `bounding_box_annotations` is a list of:
    - `display_name`: The label of the object
    - `x_min`, `y_min`, `x_max`, `y_max`: The coordinates for the bounding box

{
  "image_gcs_uri": image,
  "bounding_box_annotations": [
    {
      "display name": label,
      "x_min": "X_MIN",
      "y_min": "Y_MIN",
      "x_max": "X_MAX",
      "y_max": "Y_MAX"
      }
    },
    {
      "displayName": "OBJECT2_LABEL",
      "x_min": "X_MIN",
      "y_min": "Y_MIN",
      "x_max": "X_MAX",
      "y_max": "Y_MAX"
    }
  ]
}


For image segmentation, the JSONL index file has the requirements:

- Each data item is a separate JSON object, on a separate line.
- The key/value pair `image_gcs_uri` is the Cloud Storage path to the image.
- The key/value pair `category_mask_uri`  is the Cloud Storage path to the mask image in PNG format.
- The key/value pair `'annotation_spec_colors'` is a list mapping mask colors to a label.
  - The key/value pair pair `display_name` is the label for the pixel color mask.
  - The key/value pair pair `color` are the RGB normalized pixel values (between 0 and 1) of the mask for the corresponding label.

    { 'image_gcs_uri': image, 
      'segmentation_annotations': { 'category_mask_uri': mask_image, 'annotation_spec_colors' : [ 
          { 'display_name': label, 'color': {"red": value, "blue", value, "green": value} }, ...
      ] 
    }
    
##### ML_USE

Each JSONL object may additionally specify which split to assign the data item to when the dataset is split for training; otherwise, the dataset will be randomly split: 80/10/10.

"data_item_resource_labels": {
      "aiplatform.googleapis.com/ml_use": "training|test|validation"
    }

*Note*: The dictionary key fields may alternatively be in camelCase. For example, 'image_gcs_uri' can also be 'imageGcsUri'.

#### Location of Cloud Storage training data.

Now set the variable `IMPORT_FILE` to the location of the CSV index file in Cloud Storage.

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

#### Quick peek at your data

This tutorial uses a version of the Happy Moments dataset that is stored in a public Cloud Storage bucket, using a CSV index file.

Start by doing a quick peek at the data. You count the number of examples by counting the number of rows in the CSV index file  (`wc -l`) and then peek at the first few rows.

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `ImageDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `import_schema_uri`: The data labeling schema for the data items:
  - `single_label`: Binary and multi-class classification
  - `multi_label`: Multi-label multi-class classification
  - `bounding_box`: Object detection
  - `image_segmentation`: Segmentation

Learn more about [ImageDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-image).

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="flowers_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLImageTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: An image classification model.
  - `object_detection`: An image object detection model.
- `multi_label`: If a classification task, whether single (`False`) or multi-labeled (`True`).
- `model_type`: The type of model for deployment.
  - `CLOUD`: Deployment on Google Cloud
  - `CLOUD_HIGH_ACCURACY_1`: Optimized for accuracy over latency for deployment on Google Cloud.
  - `CLOUD_LOW_LATENCY_`: Optimized for latency over accuracy for deployment on Google Cloud.
  - `MOBILE_TF_VERSATILE_1`: Deployment on an edge device.
  - `MOBILE_TF_HIGH_ACCURACY_1`:Optimized for accuracy over latency for deployment on an edge device.
  - `MOBILE_TF_LOW_LATENCY_1`: Optimized for latency over accuracy for deployment on an edge device.
- `base_model`: (optional) Transfer learning from existing `Model` resource -- supported for image classification only.

The instantiated object is the DAG (directed acyclic graph) for the training job.

In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
    model_type="MOBILE_TF_LOW_LATENCY_1",
    base_model=None,
)

print(dag)

#### Run the training pipeline

Next, you run the created DAG to start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `validation_fraction_split`: The percentage of the dataset to use for validation.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of milli node-hours (1000 = node-hour).
- `disable_early_stopping`: If `True`, training maybe completed before using the entire budget if the service believes it cannot further improve on the model objective measurements.

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take upto > 30 minutes.

In [ ]:
if os.getenv("IS_TESTING"):
    sys.exit(0)

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

## Review model evaluation scores

After your model training has finished, you can review the evaluation scores for it using the `list_model_evaluations()` method. This method will return an iterator for each evaluation slice.

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

## Deploy the model

Next, deploy your model for online prediction. To deploy the model, you invoke the `deploy` method.

In [ ]:
endpoint = model.deploy()

## Send a online prediction request

Send a online prediction to your deployed model.

### Get test item

You will use an arbitrary example out of the dataset as a test item. Don't be concerned that the example was likely used in training the model. You are just looking at how to make a prediction.

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
if len(str(test_item[0]).split(",")) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### Make the prediction

Now that your `Model` resource is deployed to an `Endpoint` resource, you can do online predictions by sending prediction requests to the Endpoint resource.

#### Request

Since your test item is in a public Cloud Storage bucket in this example, you copy it to your bucket and read the contents of the image using `Cloud Storage SDK`. To pass the test data to the prediction service, you encode the bytes into base64 which makes the content safe from modification while transmitting binary data over the network.

The format of each instance is:

    { 'content': { 'b64': base64_encoded_bytes } }

Since the `predict()` method can take multiple items (instances), send your single test item as a list of one test item.

#### Response

The response from the `predict()` call is a Python dictionary with the following entries:

- `ids`: The internal assigned unique identifiers for each prediction request.
- `displayNames`: The class names for each class label.
- `confidences`: The predicted confidence, between 0 and 1, per class label.
- `deployed_model_id`: The Vertex AI identifier for the deployed Model resource which did the predictions.

In [ ]:
import base64

from google.cloud import storage

# Copy the test image to the Cloud storage bucket as "test.jpg"
test_image_local = "{}/test.jpg".format(BUCKET_URI)
! gsutil cp $test_item $test_image_local

# Download the test image in bytes format
storage_client = storage.Client(project=PROJECT_ID)
bucket = storage_client.bucket(bucket_name=BUCKET_NAME)
test_content = bucket.get_blob("test.jpg").download_as_bytes()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(test_content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

#### Alternate method using [GFile](https://www.tensorflow.org/api_docs/python/tf/io/gfile/GFile)

Alternatively, [GFile](https://www.tensorflow.org/api_docs/python/tf/io/gfile/GFile) method from tensorflow-io library can be used to read the data from Cloud storage directly. The following code snippet does the same :

```
import base64
import tensorflow as tf

# Read the test file using GFile
with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)
```
Nevertheless, `tf.io.gfile.GFile` supports multiple file system implementations, including local files, Google Cloud Storage (using a gs:// prefix), and HDFS (using an hdfs:// prefix).

#### Undeploy the model

When you are done doing predictions, you undeploy the model from the `Endpoint` resouce. This deprovisions all compute resources and ends billing for the deployed model.

In [ ]:
endpoint.undeploy_all()

## Export as Edge model

You can export an AutoML cloud model as a `Edge` model which you can then custom deploy to an edge device or download locally. Use the method `export_model()` to export the model to Cloud Storage, which takes the following parameters:

- `artifact_destination`: The Cloud Storage location to store the SavedFormat model artifacts to.
- `export_format_id`: The format to save the model format as. For AutoML cloud there is just one option:
   - `tf-saved-model`: TensorFlow SavedFormat for deployment to a container.
   - `tflite`: TensorFlow Lite for deployment to an edge or mobile device.
   - `edgetpu-tflite`: TensorFlow Lite for TPU
   - `tf-js`: TensorFlow for web client
   - `coral-ml`: for Coral devices

- `sync`: Whether to perform operational sychronously or asynchronously.

In [ ]:
response = model.export_model(
    artifact_destination=BUCKET_URI, export_format_id="tflite", sync=True
)

model_package = response["artifactOutputUri"]

#### Delete the model

The method 'delete()' will delete the model.

In [ ]:
model.delete()

#### Delete the dataset

The method 'delete()' will delete the dataset.

In [ ]:
dataset.delete()

#### Delete the endpoint

The method 'delete()' will delete the endpoint.

In [ ]:
endpoint.delete()

## AutoML tabular models

AutoML can train the following types of tabular models:

- classification
- regression
- forecasting

A model can be trained for either automatic deployment to the cloud or exported for manual deployment to the cloud.

Learn more about [AutoML Model Types](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)

### Data preparation

The Vertex AI `Dataset` resource for tabular has a couple of requirements for your tabular data.

- Must be in a CSV file or a BigQuery table.

Learn more about [Preparing tabular data](https://cloud.google.com/vertex-ai/docs/datasets/prepare-tabular)

#### CSV

For tabular models, the CSV file has a few requirements:

- The first row must be the heading -- note how this is different from Image, Text and Video where the requirement is no heading.
- All but one column are features.
- One column is the label, which you will specify when you subsequently create the training pipeline.

##### ML_USE

Each row may additionally specify which split to assign the data item to when the dataset is split for training; otherwise, the dataset will be randomly split: 80/10/10.

The `ml_use` assignment is specified by prepending a column for specifying the assignment -- as the first column. The value may be one of: training, test, or validation.

#### Location of BigQuery training data.

Now set the variable `IMPORT_FILE` to the location of the data table in BigQuery.

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"
BQ_TABLE = "bigquery-public-data.samples.gsod"

### Create the Dataset

#### BigQuery input data

Next, create the `Dataset` resource using the `create` method for the `TabularDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `bq_source`: Import data items from a BigQuery table into the `Dataset` resource.
- `labels`: User defined metadata. In this example, you store the location of the Cloud Storage bucket containing the user defined data.

Learn more about [TabularDataset from BigQuery table](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api#aiplatform_create_dataset_tabular_bigquery_sample-python).

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="gsod_" + TIMESTAMP,
    bq_source=[IMPORT_FILE],
    labels={"user_metadata": BUCKET_NAME},
)

label_column = "mean_temp"

print(dataset.resource_name)

In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLTabularTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `optimization_prediction_type`: The type task to train the model for.
  - `classification`: A tabuar classification model.
  - `regression`: A tabular regression model.
- `column_transformations`: (Optional): Transformations to apply to the input columns
- `optimization_objective`: The optimization objective to minimize or maximize.
  - binary classification:
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - multi-class classification:
    - `minimize-log-loss`
  - regression:
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

In [ ]:
dag = aiplatform.AutoMLTabularTrainingJob(
    display_name="gsod_" + TIMESTAMP,
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

print(dag)

#### Run the training pipeline

Next, you run the created DAG to start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `validation_fraction_split`: The percentage of the dataset to use for validation.
- `target_column`: The name of the column to train as the label.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of millihours (1000 = hour).
- `disable_early_stopping`: If `True`, training maybe completed before using the entire budget if the service believes it cannot further improve on the model objective measurements.

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take upto > 30 minutes.

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="gsod_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column="mean_temp",
)

## Review model evaluation scores

After your model training has finished, you can review the evaluation scores for it using the `list_model_evaluations()` method. This method will return an iterator for each evaluation slice.

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

## Deploy the model

Next, deploy your model for online prediction. To deploy the model, you invoke the `deploy` method, with the following parameters:

- `machine_type`: The type of compute machine.

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

#### Undeploy the model

When you are done doing predictions, you undeploy the model from the `Endpoint` resouce. This deprovisions all compute resources and ends billing for the deployed model.

In [ ]:
endpoint.undeploy_all()

## Export as cloud model

You can export an AutoML cloud model as a TensorFlow SavedFormat model which you can then custom deploy to Cloud Storage or download locally. Use the method `export_model()` to export the model to Cloud Storage, which takes the following parameters:

- `artifact_destination`: The Cloud Storage location to store the SavedFormat model artifacts to.
- `export_format_id`: The format to save the model format as. For AutoML cloud there is just one option:
   - `tf-saved-model`: TensorFlow SavedFormat
- `sync`: Whether to perform operational sychronously or asynchronously.

In [ ]:
response = model.export_model(
    artifact_destination=BUCKET_URI, export_format_id="tf-saved-model", sync=True
)

model_package = response["artifactOutputUri"]

#### Delete the model

The method 'delete()' will delete the model.

In [ ]:
model.delete()

#### Delete the dataset

The method 'delete()' will delete the dataset.

In [ ]:
dataset.delete()

#### Delete the endpoint

The method 'delete()' will delete the endpoint.

In [ ]:
endpoint.delete()

## AutoML text models

AutoML can train the following types of text models:

- classification
- sentiment analysis
- entity extraction

Learn more about [AutoML Model Types](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)

### Data preparation

The Vertex AI `Dataset` resource for text has a couple of requirements for your text data.

- Text examples must be stored in a CSV or JSONL file.

Learn more about [Preparing text data](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text)

#### CSV

For text classification, the CSV file has a few requirements:

- No heading.
- First column is the text example or Cloud Storage path to text file (.txt suffix).
- Second column the label.
- Any remaining columns are additional labels for multi-label text classification.

For text sentiment analysis, the CSV file has a few requirements:

- No heading.
- First column is the text example or Cloud Storage path to text file (.txt suffix).
- Second column is the sentiment value.
- Third column is the maximum possible sentiment value.

##### ML_USE

Each row may additionally specify which split to assign the data item to when the dataset is split for training; otherwise, the dataset will be randomly split: 80/10/10.

The `ml_use` assignment is specified by prepending a column for specifying the assignment -- as the first column. The value may be one of: training, test, or validation.

#### JSONL 

For text classification, the JSONL file has a few requirements:

- Each data item is a separate JSON object, on a separate line.
- The key/value pair `text_gcs_uri` is the Cloud Storage path to the text file.
- The key/value pair `text_content` is the alternate way of specifying the text as inlined.
- The key/value pair `display_name` is the label for the text.

{
  "classification_annotation": {
    "display_name": label
  },
  "text_content": text
}
{
  "classification_annotation": {
    "display_name": label
  },
  "text_gcs_uri": "gcs_uri_to_file"
}

    
For multi-label, the labels are specified as a list of `display_name` key/value pairs:

      'classification_annotations': [
          { 'display_name': label1
          },
          { 'display_name': labelN
          },
       ]

For text sentiment analysis, the JSONL file has a few requirements:

- Each data item is a separate JSON object, on a separate line.
- The key/value pair `text_gcs_uri` is the Cloud Storage path to the text file.
- The key/value pair `text_content` is the alternate way of specifying the text as inlined.
- The key/value pair `sentiment` is the sentiment value as an integer value greater than 0.
- The key/value pair `sentiment_max`is the maximum possible value for the sentiment.

{
  "sentiment_annotation": {
    "sentiment": number,
    "sentiment_max": number
  },
  "text_content": text,
}
{
  "sentiment_annotation": {
    "sentiment": number,
    "sentiment_max": number
  },
  "text_gcs_uri": "gcs_uri_to_file"
}


For text entity extraction, the JSONL file has a few requirements:

- Each data item is a separate JSON object, on a separate line.
- The key/value pair `text_gcs_uri` is the Cloud Storage path to the text file.
- The key/value pair `text_content` is the alternate way of specifying the text as inlined.
- The key/value pair `start_offset` is the character offset of the start of the text.
- The key/value pair `end_offset` is the character offset of the end of the text.
- The key/value pair `display_name` is the label for the text.

{
    "text_segment_annotations": [
      {
        "start_offset":number,
        "end_offset":number,
        "display_name": label
      },
      ...
    ],
    "textContent": "inline_text"
}
{
    "textSegmentAnnotations": [
      {
        "start_offset": number,
        "end_offset": number,
        "displayName": label
      },
      ...
    ],
    "text_gcs_uri": "gcs_uri_to_file"
}

##### ML_USE

Each JSONL object may additionally specify which split to assign the data item to when the dataset is split for training; otherwise, the dataset will be randomly split: 80/10/10.

"data_item_resource_labels": {
      "aiplatform.googleapis.com/ml_use": "training|test|validation"
    }

*Note*: The dictionary key fields may alternatively be in camelCase. For example, 'text_gcs_uri' can also be 'textGcsUri'.

#### Location of Cloud Storage training data.

Now set the variable `IMPORT_FILE` to the location of the CSV index file in Cloud Storage.

In [ ]:
IMPORT_FILE = "gs://cloud-ml-data/NL-classification/happiness.csv"

#### Quick peek at your data

This tutorial uses a version of the Happy Moments dataset that is stored in a public Cloud Storage bucket, using a CSV index file.

Start by doing a quick peek at the data. You count the number of examples by counting the number of rows in the CSV index file  (`wc -l`) and then peek at the first few rows.

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `TextDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `import_schema_uri`: The data labeling schema for the data items.
  - `single_label`: Binary and multi-class classification
  - `multi_label`: Multi-label multi-class classification
  - `sentiment`: Sentiment analysis
  - `extraction`: Entity extraction

Learn more about [TextDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-text).

In [ ]:
dataset = aiplatform.TextDataset.create(
    display_name="happydb_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.text.single_label_classification,
)

print(dataset.resource_name)

### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLTextTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: A text classification model.
  - `sentiment`: A text sentiment analysis model.
  - `extraction`: A text entity extraction model.
- `multi_label`: If a classification task, whether single (False) or multi-labeled (True).
- `sentiment_max`: If a sentiment analysis task, the maximum sentiment value.


In [ ]:
dag = aiplatform.AutoMLTextTrainingJob(
    display_name="happydb_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
)

print(dag)

#### Run the training pipeline

Next, you run the created DAG to start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `validation_fraction_split`: The percentage of the dataset to use for validation.

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take upto > 30 minutes.

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="happydb_" + TIMESTAMP,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
)

## Review model evaluation scores

After your model training has finished, you can review the evaluation scores for it using the `list_model_evaluations()` method. This method will return an iterator for each evaluation slice.

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

## Deploy the model

Next, deploy your model for online prediction. To deploy the model, you invoke the `deploy` method.

In [ ]:
endpoint = model.deploy()

#### Undeploy the model

When you are done doing predictions, you undeploy the model from the `Endpoint` resouce. This deprovisions all compute resources and ends billing for the deployed model.

In [ ]:
endpoint.undeploy_all()

#### Delete the model

The method 'delete()' will delete the model.

In [ ]:
model.delete()

#### Delete the dataset

The method 'delete()' will delete the dataset.

In [ ]:
dataset.delete()

#### Delete the endpoint

The method 'delete()' will delete the endpoint.

In [ ]:
endpoint.delete()

## AutoML video models

AutoML can train the following types of video models:

- classification
- object tracking
- action recognition

A model can be trained for either deployment to the cloud or exported to the edge.

Learn more about [AutoML Model Types](https://cloud.google.com/vertex-ai/docs/start/automl-model-types)

### Data preparation

The Vertex AI `Dataset` resource for text has a couple of requirements for your text data.

- Text examples must be stored in a CSV or JSONL file.

Learn more about [Preparing video data](https://cloud.google.com/vertex-ai/docs/datasets/prepare-video)

#### CSV

For video classification, the CSV file has a few requirements:

- No heading.
- First column is the Cloud Storage path to video file.
- Second column the label.
- Third column is the start time (seconds) in the video to classify.
- Fourth column is the end time (seconds) in the video to classify.

For multi-label classification, each label is a separate row entry.

For video object tracking, the CSV file has a few requirements:

- No heading.
- First column is the Cloud Storage path to video file.
- Second column the label.
- Third column is unused (blank).
- Fourth column is the start time (seconds) in the video to start tracking the object.
- The fifth through eighth columns are the vertices of the object to track.
    - x_min
    - y_min
    - x_max
    - y_max
    
For action recognition, the CSV file has a few requirements:

- No heading.
- Each row can be one of the following four formats:

VIDEO_URI, TIME_SEGMENT_START, TIME_SEGMENT_END, LABEL, ANNOTATION_FRAME_TIMESTAMP

VIDEO_URI, , , LABEL, ANNOTATION_FRAME_TIMESTAMP

VIDEO_URI, TIME_SEGMENT_START, TIME_SEGMENT_END, LABEL, ANNOTATION_SEGMENT_START, ANNOTATION_SEGMENT_END

VIDEO_URI, , , LABEL, ANNOTATION_SEGMENT_START, ANNOTATION_SEGMENT_END


##### ML_USE

Each row may additionally specify which split to assign the data item to when the dataset is split for training; otherwise, the dataset will be randomly split: 80/10/10.

The `ml_use` assignment is specified by prepending a column for specifying the assignment -- as the first column. The value may be one of: training, or test.

#### JSONL

For video classification, the CSV file has a few requirements:

- Each data item is a separate JSON object, on a separate line.
- The key/value pair `video_gcs_uri` is the Cloud Storage path to the text file.
- The key/value pair `display_name` is the label for the text.
- The key/value pair `start_time` is the start time (seconds) for classifying.
- The key/value pair `end_time` is the end time (seconds) for classifying.


    {
        "video_gcs_uri": video,
        "time_segment_annotations": [{
            "display_name": label,
            "start_time": "start_time_of_segment",
            "end_time": "end_time_of_segment"
        }]
    }

For video object tracking, the CSV file has a few requirements:

- Each data item is a separate JSON object, on a separate line.
- The key/value pair `video_gcs_uri` is the Cloud Storage path to the text file.

    {
        "video_gcs_uri": video,
        "temporal_bounding_box_annotations": [{
            "display_name": label,
            "x_min": "leftmost_coordinate_of_the_bounding box",
            "x_max": "rightmost_coordinate_of_the_bounding box",
            "y_min": "topmost_coordinate_of_the_bounding box",
            "y_max": "bottommost_coordinate_of_the_bounding box",
            "time_offset": "timeframe_object-detected"
        }]
    }

For video action recognition, the CSV file has a few requirements:

- Each data item is a separate JSON object, on a separate line.
- The key/value pair `video_gcs_uri` is the Cloud Storage path to the text file.

    {
      "video_gcs_uri': video,
      "time_segments": [{
        "start_time": "start_time_of_fully_annotated_segment",
        "end_time": "end_time_of_segment"}],
      "time_segment_annotations": [{
        "display_name": label,
        "start_time": "start_time_of_segment",
        "end_time": "end_time_of_segment"
      }]
    }

##### ML_USE

Each JSONL object may additionally specify which split to assign the data item to when the dataset is split for training; otherwise, the dataset will be randomly split: 80/20.

"data_item_resource_labels": {
      "aiplatform.googleapis.com/ml_use": "training|test"
    }

*Note*: The dictionary key fields may alternatively be in camelCase. For example, 'video_gcs_uri' can also be 'videoGcsUri'.

#### Location of Cloud Storage training data.

Now set the variable `IMPORT_FILE` to the location of the CSV index file in Cloud Storage.

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/hmdb_split1_5classes_train_inf.csv"

#### Quick peek at your data

This tutorial uses a version of the Happy Moments dataset that is stored in a public Cloud Storage bucket, using a CSV index file.

Start by doing a quick peek at the data. You count the number of examples by counting the number of rows in the CSV index file  (`wc -l`) and then peek at the first few rows.

In [ ]:
FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `VideoDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `import_schema_uri`: The data labeling schema for the data items.
  - `classification`: Binary and multi-class classification
  - `object_tracking`: Object tracking
  - `action_recognition`: Action recognition

Learn more about [VideoDataset](https://cloud.google.com/vertex-ai/docs/datasets/prepare-video).

In [ ]:
dataset = aiplatform.VideoDataset.create(
    display_name="human_motion_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.video.classification,
)

print(dataset.resource_name)

### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLVideoTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: A video classification model.
  - `object_tracking`: A video object tracking model.
  - `action_recognition`: A video action recognition model.

In [ ]:
dag = aiplatform.AutoMLVideoTrainingJob(
    display_name="human_motion_" + TIMESTAMP,
    prediction_type="classification",
)

print(dag)

#### Run the training pipeline

Next, you run the created DAG to start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take upto > 30 minutes.

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="human_motion_" + TIMESTAMP,
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

## Review model evaluation scores

After your model training has finished, you can review the evaluation scores for it using the `list_model_evaluations()` method. This method will return an iterator for each evaluation slice.

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

#### Delete the model

The method 'delete()' will delete the model.

In [ ]:
model.delete()

#### Delete the dataset

The method 'delete()' will delete the dataset.

In [ ]:
dataset.delete()

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI